In [191]:
import pandas as pd
import os 
import numpy as np

In [193]:
def column_name_converter(string):
    return string.lower().replace("\n", "_").replace(" ", "_")

In [194]:
def change_column_names(df):
    column_name0 = list(df.columns)
    column_name1 =[column_name_converter(column) for column in column_name0]
    column_name2 = list(df.iloc[0])
    assert len(column_name1) == len(column_name2)
    
    prefix = ''
    for i, name in enumerate(column_name2):
        if isinstance(name, str):
            if prefix != column_name1[i] and 'unnamed:' not in column_name1[i]:
                prefix = column_name1[i]
            new_name = prefix + "_" + column_name_converter(name)
            column_name2[i] = new_name
        else:
            column_name2[i] = None

    column_name3 = []
    for i, name in enumerate(column_name2):
        if name is not None:
            column_name3.append(name)
        else:
            column_name3.append(column_name1[i])            
    
    df = df.rename(columns={column_name0[i]: column_names[i] for i in range(len(column_names))})
    df = df.drop(0)
    df = df.sort_values(by='date').reset_index()
    del df['index']
    return df

In [197]:
def rule_closeup(df):
    """
        returns a DataFrame column that is true (1) if the current dates 
        close_price is higher than the close_price of the previous date
    """
    assert 'close_price' in list(df.columns)
    assert 'date' in list(df.columns)
    
    closeup = [None]
    for i in range(1, len(df.index)):
        previous_close_price = df.iloc[i-1]['close_price']
        current_close_price = df.iloc[i]['close_price']        
        closeup.append(current_close_price > previous_close_price)
    assert len(closeup) == len(df.index)
    return closeup
        

In [217]:
df = pd.read_excel('res/Euro.xls')
df = change_column_names(df)
closeup = rule_closeup(df)
closeup_df = pd.DataFrame(data=closeup, columns=['rule_closeup'])
df = pd.concat([df, closeup_df], axis=1)